<a href="https://colab.research.google.com/github/Shafna81/fakenewsdetection/blob/main/12RoBERTa%2C_adam%2C_2E_5%2C_bs_16%2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import time

In [2]:
!pip install transformers
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
from torch.optim import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

2025-05-20 03:54:35.562388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747713275.739705      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747713275.792341      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 12
torch.manual_seed(seed)

In [4]:
# df = pd.read_csv('datacleaned.csv')
# gosi = pd.read_csv("/kaggle/input/fakenewcombined/gossipcop_train.csv")
# liar = pd.read_csv("/kaggle/input/fakenewcombined/liar_train_covid_format.csv")
# poltifact = pd.read_csv("/kaggle/input/fakenewcombined/politifact_train.csv")
# print(gosi.head(3))
# print(liar.head(3))
# print(poltifact.head(3))

import pandas as pd

# 1. Đọc các file CSV
gosi      = pd.read_csv("/kaggle/input/fakenewcombined/gossipcop_train.csv")
liar      = pd.read_csv("/kaggle/input/fakenewcombined/liar_train_covid_format.csv")
politifact = pd.read_csv("/kaggle/input/fakenewcombined/politifact_train.csv")

gosi_test = pd.read_csv("/kaggle/input/fakenewcombined/gossipcop_test.csv")
liar_test = pd.read_csv("/kaggle/input/fakenewcombined/liar_test_covid_format.csv")
politifact_test = pd.read_csv("/kaggle/input/fakenewcombined/politifact_test.csv")

gosi_val = pd.read_csv("/kaggle/input/fakenewcombined/gossipcop_val.csv")
liar_val = pd.read_csv("/kaggle/input/fakenewcombined/liar_valid_covid_format.csv")
politifact_val = pd.read_csv("/kaggle/input/fakenewcombined/politifact_val.csv")
# 2. Viết hàm chuẩn hoá: chọn 2 cột tweet & label, đổi tên, và map label sang số
def prepare(df):
    df = df[['tweet', 'label']].copy()
    # rename
    df.columns = ['Text', 'Label']
    # map giá trị: 'real' -> 1, 'fake' -> 0
    df['Label'] = df['Label'].map({'real': 1, 'fake': 0})
    return df

# 3. Áp dụng cho từng DataFrame
gosi_df_train      = prepare(gosi)
liar_df_train      = prepare(liar)
politifact_df_train = prepare(politifact)

gosi_test_df = prepare(gosi_test)
liar_test_df = prepare(liar_test)
politifact_test_df = prepare(politifact_test)

gosi_val_df = prepare(gosi_val)
liar_val_df = prepare(liar_val)
politifact_val_df = prepare(politifact_val)

# 4. Gộp lại và reset index
combined_df_train = pd.concat([gosi_df_train, liar_df_train, politifact_df_train], ignore_index=True)
combined_df_test = pd.concat([gosi_test_df, liar_test_df, politifact_test_df], ignore_index=True)
combined_df_val = pd.concat([gosi_val_df, liar_val_df, politifact_val_df], ignore_index=True)

print(combined_df_train.shape) 
print(combined_df_train.head()) 

combined_df_train.to_csv("combined_data_train.csv", index=False)
combined_df_test.to_csv("combined_data_test.csv", index=False)
combined_df_val.to_csv("combined_data_val.csv", index=False)

(26477, 2)
                                                Text  Label
0  Kanye West Surprises Kim Kardashian With Cloud...      1
1  Hollywood Mourns Death of ‘Legendary’ Harry De...      0
2  Eric Dane Responds to Rebecca Gayheart's Divor...      1
3     Were James Garner and Jennifer Garner related?      1
4  Shailene Woodley Makes Her Red Carpet Debut wi...      1


In [5]:
df = pd.read_csv("/kaggle/working/combined_data_train.csv")
df.head()

,Text,Label
0,Kanye West Surprises Kim Kardashian With Cloud...,1
1,Hollywood Mourns Death of ‘Legendary’ Harry De...,0
2,Eric Dane Responds to Rebecca Gayheart's Divor...,1
3,Were James Garner and Jennifer Garner related?,1
4,Shailene Woodley Makes Her Red Carpet Debut wi...,1


In [6]:
# Bagi dataset menjadi data training dan data testing
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenisasi data menggunakan tokenizer dari Roberta
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_encodings = tokenizer(train_df['Text'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True, max_length=512)

# Konversi data ke dalam format tensor
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']),
                                               torch.tensor(train_df['Label'].tolist()))

test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                              torch.tensor(test_encodings['attention_mask']),
                                              torch.tensor(test_df['Label'].tolist()))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [7]:
# Definisikan Model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
dropout_rate = 0.5
model.config.hidden_dropout_prob = dropout_rate
model.config.attention_probs_dropout_prob = dropout_rate

In [9]:
import os
import time
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# ===================== CONFIG =====================
datasets = {
    "covid": "/kaggle/input/fakecovid/Constraint_Train.csv",
    "gossip": "/kaggle/input/fakenewcombined/gossipcop_train.csv",
    "politifact": "/kaggle/input/fakenewcombined/politifact_train.csv",
    "liar": "/kaggle/input/fakenewcombined/liar_train_covid_format.csv"
}

base_model = "roberta-base"
output_base = "/kaggle/working/checkpoints"
num_epochs = 5
batch_size = 16
max_length = 256
learning_rate = 2e-5

# ===================== DATA PREP =====================
def prepare_dataframe(df):
    df = df[['tweet', 'label']].copy()
    df.columns = ['Text', 'Label']
    df['Label'] = df['Label'].map({'real': 1, 'fake': 0})
    return df

def encode_data(df, tokenizer):
    encodings = tokenizer(list(df['Text']), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    labels = torch.tensor(df['Label'].values)
    return TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)

# ===================== TRAINING =====================
def train_model_on_dataset(model, tokenizer, dataset_path, dataset_name):
    print(f"\n===== Training on {dataset_name} =====")
    
    df = pd.read_csv(dataset_path)
    df = prepare_dataframe(df)
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=seed, stratify=df['Label'])
    
    train_data = encode_data(train_df, tokenizer)
    val_data = encode_data(val_df, tokenizer)
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs} on {dataset_name}")
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attn_mask, labels = batch

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attn_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            total_loss += loss.item()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        print(f"Train Loss: {total_loss / len(train_loader):.4f}")
        
        # Eval
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = tuple(t.to(device) for t in batch)
                input_ids, attn_mask, labels = batch
                outputs = model(input_ids, attention_mask=attn_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        acc = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)
        print(f"Validation Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

    # Save checkpoint
    output_path = os.path.join(output_base, f"roberta_finetuned_on_{dataset_name}")
    os.makedirs(output_path, exist_ok=True)
    model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    print(f"✅ Saved model checkpoint at: {output_path}")

    return output_path

# ===================== MAIN LOOP =====================
# Khởi tạo tokenizer và model lần đầu từ 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(base_model)
model = RobertaForSequenceClassification.from_pretrained(base_model, num_labels=2)
model.to(device)

# Train lần lượt trên từng dataset, update model sau mỗi bước
for dataset_name, dataset_path in datasets.items():
    checkpoint_path = train_model_on_dataset(model, tokenizer, dataset_path, dataset_name)
    # Sau khi train xong 1 tập -> load lại từ checkpoint mới nhất (tiếp tục fine-tune)
    tokenizer = RobertaTokenizer.from_pretrained(checkpoint_path)
    model = RobertaForSequenceClassification.from_pretrained(checkpoint_path, num_labels=2)
    model.to(device)

print("\n🎉 All datasets trained and checkpoints saved!")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Training on covid =====

Epoch 1/5 on covid
Train Loss: 0.2406
Validation Accuracy: 0.9696 | F1 Score: 0.9708

Epoch 2/5 on covid
Train Loss: 0.1021
Validation Accuracy: 0.9688 | F1 Score: 0.9708

Epoch 3/5 on covid
Train Loss: 0.0371
Validation Accuracy: 0.9766 | F1 Score: 0.9778

Epoch 4/5 on covid
Train Loss: 0.0212
Validation Accuracy: 0.9751 | F1 Score: 0.9760

Epoch 5/5 on covid
Train Loss: 0.0066
Validation Accuracy: 0.9743 | F1 Score: 0.9756
✅ Saved model checkpoint at: /kaggle/working/checkpoints/roberta_finetuned_on_covid

===== Training on gossip =====

Epoch 1/5 on gossip
Train Loss: 0.4388
Validation Accuracy: 0.8606 | F1 Score: 0.9128

Epoch 2/5 on gossip
Train Loss: 0.3076
Validation Accuracy: 0.8581 | F1 Score: 0.9104

Epoch 3/5 on gossip
Train Loss: 0.2362
Validation Accuracy: 0.8503 | F1 Score: 0.9038

Epoch 4/5 on gossip
Train Loss: 0.1900
Validation Accuracy: 0.8490 | F1 Score: 0.9014

Epoch 5/5 on gossip
Train Loss: 0.1548
Validation Accuracy: 0.8542 | F1 Sc

In [10]:
# test model combined với từng tập dataset
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset

# 1. Thiết lập device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Load tokenizer & model của combined
model_dir = "/kaggle/working/checkpoints/roberta_finetuned_on_liar"  # đường dẫn tới folder chứa từng file config.json, pytorch_model.bin, ...
tokenizer = RobertaTokenizer.from_pretrained(model_dir)
model = RobertaForSequenceClassification.from_pretrained(model_dir)
model.to(device)
model.eval()

# 3. Hàm chuẩn hoá DataFrame test
def prepare(df):
    df = df[['tweet', 'label']].copy()
    df.columns = ['Text', 'Label']
    df['Label'] = df['Label'].map({'real': 1, 'fake': 0})
    return df

# 4. Hàm đánh giá trên một DataFrame đã chuẩn hoá
def evaluate(df, batch_size=16, max_length=512):
    texts = df['Text'].tolist()
    labels = df['Label'].tolist()
    # Tokenize
    enc = tokenizer(texts,
                    padding=True,
                    truncation=True,
                    max_length=max_length,
                    return_tensors='pt')
    dataset = TensorDataset(enc['input_ids'], enc['attention_mask'], torch.tensor(labels))
    loader = DataLoader(dataset, batch_size=batch_size)

    all_preds, all_labels = [], []
    with torch.no_grad():
        for input_ids, attention_mask, batch_labels in loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            logits = model(input_ids, attention_mask=attention_mask).logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(batch_labels.numpy())

    acc   = accuracy_score(all_labels, all_preds)
    prec  = precision_score(all_labels, all_preds, zero_division=0)
    rec   = recall_score(all_labels, all_preds, zero_division=0)
    f1    = f1_score(all_labels, all_preds, zero_division=0)
    return acc, prec, rec, f1

# 5. Đường dẫn tới các file test CSV gốc
datasets = {
    "GossipCop":    "/kaggle/input/fakenewcombined/gossipcop_test.csv",
    "Liar":         "/kaggle/input/fakenewcombined/liar_test_covid_format.csv",
    "PolitiFact":   "/kaggle/input/fakenewcombined/politifact_test.csv",
    "FakeCovid": "/kaggle/input/fakecovid/Constraint_Val.csv",
    # nếu bạn muốn test luôn trên combined_data_test.csv:
    # "Combined":     "combined_data_test.csv"
}

# 6. Chạy evaluate lần lượt
for name, path in datasets.items():
    df_raw = pd.read_csv(path)
    df_test = prepare(df_raw)
    acc, prec, rec, f1 = evaluate(df_test)
    print(f"{name} Test set:")
    print(f"  → Accuracy : {acc:.4f}")
    print(f"  → Precision: {prec:.4f}")
    print(f"  → Recall   : {rec:.4f}")
    print(f"  → F1 Score : {f1:.4f}\n")


GossipCop Test set:
  → Accuracy : 0.3391
  → Precision: 0.9504
  → Recall   : 0.1368
  → F1 Score : 0.2392

Liar Test set:
  → Accuracy : 0.6535
  → Precision: 0.5103
  → Recall   : 0.5523
  → F1 Score : 0.5305

PolitiFact Test set:
  → Accuracy : 0.5597
  → Precision: 0.9286
  → Recall   : 0.2766
  → F1 Score : 0.4262

FakeCovid Test set:
  → Accuracy : 0.8313
  → Precision: 0.8717
  → Recall   : 0.7946
  → F1 Score : 0.8314



In [11]:
import os
import time
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ======================= CONFIG ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

datasets = {
    "covid": "/kaggle/input/fakecovid/Constraint_Train.csv",
    "gossip": "/kaggle/input/fakenewcombined/gossipcop_train.csv",
    "politifact": "/kaggle/input/fakenewcombined/politifact_train.csv",
    "liar": "/kaggle/input/fakenewcombined/liar_train_covid_format.csv"
}

output_base = "/kaggle/working/roberta_from_scratch"
num_epochs = 5
batch_size = 16
max_length = 256
learning_rate = 2e-5
base_model = "roberta-base"

# =================== HELPER FUNCTIONS ===================
def prepare_dataframe(df):
    df = df[['tweet', 'label']].copy()
    df.columns = ['Text', 'Label']
    df['Label'] = df['Label'].map({'real': 1, 'fake': 0})
    return df

def encode_data(df, tokenizer):
    encodings = tokenizer(list(df['Text']), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    labels = torch.tensor(df['Label'].values)
    return TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)

def evaluate_model(model, val_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attn_mask, labels = batch
            outputs = model(input_ids, attention_mask=attn_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    return acc, prec, rec, f1

# =================== MAIN TRAINING LOOP ===================
for dataset_name, dataset_path in datasets.items():
    print(f"\n🚀 Training fresh model on: {dataset_name}")

    # Load & prepare
    df = pd.read_csv(dataset_path)
    df = prepare_dataframe(df)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed, stratify=df['Label'])

    # Tokenizer + Dataset
    tokenizer = RobertaTokenizer.from_pretrained(base_model)
    train_data = encode_data(train_df, tokenizer)
    test_data = encode_data(test_df, tokenizer)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Init fresh model
    model = RobertaForSequenceClassification.from_pretrained(base_model, num_labels=2)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Training
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attn_mask, labels = batch
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attn_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            total_loss += loss.item()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        print(f"Training Loss: {total_loss / len(train_loader):.4f}")

    # Evaluation
    acc, prec, rec, f1 = evaluate_model(model, test_loader)
    print(f"\n📊 Evaluation on test set of {dataset_name}:")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")

    # Save checkpoint
    model_dir = os.path.join(output_base, f"{dataset_name}_checkpoint")
    os.makedirs(model_dir, exist_ok=True)
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    print(f"✅ Model saved at: {model_dir}")

print("\n✅✅ All models trained separately from scratch and saved.")



🚀 Training fresh model on: covid


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/5
Training Loss: 0.2406

Epoch 2/5
Training Loss: 0.0970

Epoch 3/5
Training Loss: 0.0368

Epoch 4/5
Training Loss: 0.0232

Epoch 5/5
Training Loss: 0.0092

📊 Evaluation on test set of covid:
Accuracy : 0.9790
Precision: 0.9736
Recall   : 0.9866
F1 Score : 0.9800
✅ Model saved at: /kaggle/working/roberta_from_scratch/covid_checkpoint

🚀 Training fresh model on: gossip


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/5
Training Loss: 0.4156

Epoch 2/5
Training Loss: 0.3168

Epoch 3/5
Training Loss: 0.2435

Epoch 4/5
Training Loss: 0.1945

Epoch 5/5
Training Loss: 0.1537

📊 Evaluation on test set of gossip:
Accuracy : 0.8610
Precision: 0.8988
Recall   : 0.9206
F1 Score : 0.9096
✅ Model saved at: /kaggle/working/roberta_from_scratch/gossip_checkpoint

🚀 Training fresh model on: politifact


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/5
Training Loss: 0.4912

Epoch 2/5
Training Loss: 0.2477

Epoch 3/5
Training Loss: 0.1673

Epoch 4/5
Training Loss: 0.1326

Epoch 5/5
Training Loss: 0.0825

📊 Evaluation on test set of politifact:
Accuracy : 0.9189
Precision: 0.9130
Recall   : 0.9545
F1 Score : 0.9333
✅ Model saved at: /kaggle/working/roberta_from_scratch/politifact_checkpoint

🚀 Training fresh model on: liar


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/5
Training Loss: 0.6364

Epoch 2/5
Training Loss: 0.5941

Epoch 3/5
Training Loss: 0.5147

Epoch 4/5
Training Loss: 0.4110

Epoch 5/5
Training Loss: 0.3205

📊 Evaluation on test set of liar:
Accuracy : 0.6367
Precision: 0.4899
Recall   : 0.5330
F1 Score : 0.5105
✅ Model saved at: /kaggle/working/roberta_from_scratch/liar_checkpoint

✅✅ All models trained separately from scratch and saved.


In [12]:
import os
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Test set mapping
datasets = {
    "GossipCop":    "/kaggle/input/fakenewcombined/gossipcop_test.csv",
    "Liar":         "/kaggle/input/fakenewcombined/liar_test_covid_format.csv",
    "PolitiFact":   "/kaggle/input/fakenewcombined/politifact_test.csv",
    "FakeCovid":    "/kaggle/input/fakecovid/Constraint_Val.csv"
}

# 2. Load & preprocess helper
def prepare_dataframe(df):
    df = df[['tweet', 'label']].copy()
    df.columns = ['Text', 'Label']
    df['Label'] = df['Label'].map({'real': 1, 'fake': 0})
    return df

def encode_data(df, tokenizer, max_length=256):
    enc = tokenizer(df['Text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    labels = torch.tensor(df['Label'].tolist())
    return TensorDataset(enc['input_ids'], enc['attention_mask'], labels)

def evaluate(model, dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(true_labels, predictions)
    prec = precision_score(true_labels, predictions, zero_division=0)
    rec = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)
    return acc, prec, rec, f1

# 3. Test từng model tương ứng
results = []
for dataset_name, test_path in datasets.items():
    print(f"\n🔍 Testing model trained on {dataset_name}")
    checkpoint_path = f"/kaggle/working/roberta_from_scratch/{dataset_name.lower()}_checkpoint"

    # Load model + tokenizer
    tokenizer = RobertaTokenizer.from_pretrained(checkpoint_path)
    model = RobertaForSequenceClassification.from_pretrained(checkpoint_path)
    model.to(device)

    # Load test data
    df_test = pd.read_csv(test_path)
    df_test = prepare_dataframe(df_test)
    dataset = encode_data(df_test, tokenizer)
    test_loader = DataLoader(dataset, batch_size=16)

    # Evaluate
    acc, prec, rec, f1 = evaluate(model, test_loader)
    print(f"✅ Accuracy:  {acc:.4f}")
    print(f"✅ Precision: {prec:.4f}")
    print(f"✅ Recall:    {rec:.4f}")
    print(f"✅ F1 Score:  {f1:.4f}")

    results.append({
        "Dataset": dataset_name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1
    })

# 4. Summary table
summary_df = pd.DataFrame(results)
print("\n📊 Summary of test results:")
print(summary_df)

# 5. Save CSV if needed
summary_df.to_csv("/kaggle/working/evaluation_summary.csv", index=False)
print("\n📁 Saved evaluation results to /kaggle/working/evaluation_summary.csv")



🔍 Testing model trained on GossipCop


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/working/roberta_from_scratch/gossipcop_checkpoint'. Use `repo_type` argument if needed.